<a href="https://colab.research.google.com/github/yasmienAshraf/Project1/blob/main/ArabicSentimentAnalysisSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
!cp "/content/drive/My Drive/Colab Notebooks/All_Training_80%.csv" "All_Training_80%.csv"

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_original = pd.read_csv('All_Training_80%.csv')
df_original.head()

,Tweet,Polarity
0,بعد ابلاغه بصدور التصديق بحكم الاعدام ضده كان...,pos
1,بمزيدا من الفخر والاعتزاز تزف شيعيه امير المؤ...,pos
2,توقيت اعدام نمر النمر عيد راس السنه لغير المس...,pos
3,خلفه رجال الشيخ_النمر_شهيدا الشيخ_النمر القطي...,pos
4,رحم الشيخ الشهيد النمر,pos


In [44]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45436 entries, 0 to 45435
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tweet     45436 non-null  object
 1   Polarity  45436 non-null  object
dtypes: object(2)
memory usage: 710.1+ KB


In [45]:
df_original.Polarity.value_counts()

,count
Polarity,
neg,16587
neut,14985
pos,13864


In [46]:
df_original['Polarity'] = df_original['Polarity'].map({'neg': 0, 'pos': 1, 'neut': 2})

In [47]:
repeat=[]
list_of_tweets = df_original['Tweet'].to_list()
for i in range(len(list_of_tweets)-1):
  if list_of_tweets[i] in list_of_tweets[i+1:]:
    repeat.append(i)
len(repeat)

1122

In [48]:
df_original.drop(repeat,axis=0,inplace=True)
len(df_original)

44314

In [49]:
df_original.reset_index(drop=True, inplace=True)
df_original.head()

,Tweet,Polarity
0,بعد ابلاغه بصدور التصديق بحكم الاعدام ضده كان...,1
1,توقيت اعدام نمر النمر عيد راس السنه لغير المس...,1
2,خلفه رجال الشيخ_النمر_شهيدا الشيخ_النمر القطي...,1
3,رحم الشيخ الشهيد النمر,1
4,عضو المجلس القومي لحقوق الانسان في مصر كمال ا...,1


In [69]:
df_original['pre_clean_len'] = [len(t) for t in df_original.Tweet] #Number of caracters (the length) in each post
df_original


,Tweet,Polarity,pre_clean_len
0,بعد ابلاغه بصدور التصديق بحكم الاعدام ضده كان...,1,114
1,توقيت اعدام نمر النمر عيد راس السنه لغير المس...,1,112
2,خلفه رجال الشيخ_النمر_شهيدا الشيخ_النمر القطي...,1,106
3,رحم الشيخ الشهيد النمر,1,23
4,عضو المجلس القومي لحقوق الانسان في مصر كمال ا...,1,117
...,...,...,...
44309,ابدعتي,1,7
44310,احسنتي,1,7
44311,اول جمعه في عامنا هذا اللهم اسق امانينا بماا ...,1,118
44312,اي والله لها معي سنه او اكثر ممتازه جدا,1,40


In [51]:
from pprint import pprint
data_dict = {
    'polarity': {
        'type': df_original.Polarity.dtype,
        'description': 'sentiment class - 0: negative, 1: positive, 2: neutral'
    },
    'tweet': {
        'type': df_original.Tweet.dtype,
        'description': 'tweet text'
    },
    'pre_clean_len': {
        'type': df_original.pre_clean_len.dtype,
        'description': 'length of the tweet before cleaning'
   },
    'dataset.shape': df_original.shape
}

pprint(data_dict)

{'dataset.shape': (44314, 3),
 'polarity': {'description': 'sentiment class - 0: negative, 1: positive, 2: '
                             'neutral',
              'type': dtype('int64')},
 'pre_clean_len': {'description': 'length of the tweet before cleaning',
                   'type': dtype('int64')},
 'tweet': {'description': 'tweet text', 'type': dtype('O')}}


In [52]:
# Data cleaning function definition

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

In [53]:
import re
from bs4 import BeautifulSoup

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
arabic_num_pat = '[٠١٢٣٤٥٦٧٨٩]'
eng_num_pat = '[0123456789]'
sharta_pat = '[_]'
#sharta_pat = '_[أ-ي]+' #for removing hashtag
eng_pat = '[A-Za-z]'

def tweet_cleaner(text):
  soup = BeautifulSoup(text, 'lxml')
  souped = soup.get_text()
  try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
  except:
        bom_removed = souped
  stripped = re.sub(combined_pat, '', bom_removed)
  stripped = re.sub(www_pat, '', stripped)
  stripped = re.sub(arabic_num_pat, '', stripped)
  stripped = re.sub(eng_num_pat, '', stripped)
  #stripped = re.sub(r'\w+_\s?','_',stripped) #for removing hashtag
  stripped = re.sub(sharta_pat, ' ', stripped)
  stripped = re.sub(eng_pat, ' ', stripped)

  words = [x for x in tok.tokenize(stripped) if len(x) > 1]
  return (" ".join(words)).strip()

In [54]:
clean_tweet_texts = []
for i in range(0, len(df_original)):
  if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed"%(i+1,len(df_original)))
  clean_tweet_texts.append(tweet_cleaner(df_original['Tweet'][i]))

Tweets 10000 of 44314 has been processed
Tweets 20000 of 44314 has been processed
Tweets 30000 of 44314 has been processed
Tweets 40000 of 44314 has been processed


In [55]:
#normalization
def normalizeArabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "و", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)
for i in range(len(clean_tweet_texts)):
  clean_tweet_texts[i] = normalizeArabic(clean_tweet_texts[i])

In [56]:
clean_df = pd.DataFrame(clean_tweet_texts, columns=['text'])
clean_df['target'] = df_original.Polarity
clean_df.to_csv('/content/drive/My Drive/clean_tweet.csv', encoding='utf-8')

In [ ]:
!cp /content/drive/My\ Drive/clean_tweet.csv /content/

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlinBackend.figure_format = 'retina'

In [58]:
csv = '/content/drive/My Drive/clean_tweet.csv'
df_clean = pd.read_csv(csv, index_col=0)
df_clean.head()

,text,target
0,بعد ابلاغه بصدور التصديق بحكم الاعدام ضده كان ...,1
1,توقيت اعدام نمر النمر عيد راس السنه لغير المسل...,1
2,خلفه رجال الشيخ النمر شهيدا الشيخ النمر القطيف...,1
3,رحم الشيخ الشهيد النمر,1
4,عضو المجلس القومي لحقوق الانسان في مصر كمال ال...,1


In [59]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44314 entries, 0 to 44313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44314 non-null  object
 1   target  44314 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [60]:
text=df_clean['text'].copy()
target = df_clean['target'].copy()
my_df= pd.DataFrame(text,columns=['text'])
my_df['target'] = target
my_df.head()

,text,target
0,بعد ابلاغه بصدور التصديق بحكم الاعدام ضده كان ...,1
1,توقيت اعدام نمر النمر عيد راس السنه لغير المسل...,1
2,خلفه رجال الشيخ النمر شهيدا الشيخ النمر القطيف...,1
3,رحم الشيخ الشهيد النمر,1
4,عضو المجلس القومي لحقوق الانسان في مصر كمال ال...,1


In [61]:
my_df.dropna(inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44314 entries, 0 to 44313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44314 non-null  object
 1   target  44314 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [62]:
x = my_df.text
y = my_df.target

In [63]:
x = my_df.text
y = my_df.target

In [64]:
from sklearn.model_selection import train_test_split
SEED = 666
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.2, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)


In [65]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [66]:
print("Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive, {3:.2f}% neutral".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 2]) / (len(x_train)*1.))*100))
print("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive, {3:.2f}% neutral".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 2]) / (len(x_validation)*1.))*100))
print("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive, {3:.2f}% neutral".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 2]) / (len(x_test)*1.))*100))


Train set has total 35451 entries with 36.00% negative, 30.57% positive, 33.42% neutral
Validation set has total 4431 entries with 38.46% negative, 29.77% positive, 31.78% neutral
Test set has total 4432 entries with 36.44% negative, 30.39% positive, 33.17% neutral


In [67]:
%%time

param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}
clf = Pipeline([('tfidf', TfidfVectorizer()),
                ('grid',GridSearchCV(SVC(),param_grid,refit=True,verbose=3)),

])
clf.fit(x_validation,y_validation)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.387 total time=   4.4s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.387 total time=   4.9s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.387 total time=   4.8s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.388 total time=   3.7s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.387 total time=   3.7s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.384 total time=   4.1s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.385 total time=   4.4s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.385 total time=   3.6s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.385 total time=   3.6s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.384 total time=   4.1s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.384 total time=   4.1s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('grid',
                 GridSearchCV(estimator=SVC(),
                              param_grid={'C': [0.1, 1, 10, 100, 1000],
                                          'gamma': [1, 0.1, 0.01, 0.001,
                                                    0.0001],
                                          'kernel': ['rbf']},
                              verbose=3))])

In [70]:
%%time

best_clf = Pipeline([('tfidf', TfidfVectorizer()),
                ('svm',SVC(C=1000, gamma=1)),

])

best_clf.fit(x_train,y_train)

CPU times: user 16min 29s, sys: 1.99 s, total: 16min 31s
Wall time: 16min 45s


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('svm', SVC(C=1000, gamma=1))])

In [71]:
best_clf.score(x_validation,y_validation)

0.7637102234258633

In [72]:
from sklearn.metrics import accuracy_score, classification_report
pred = best_clf.predict(x_test)
accuracy_score(pred, y_test)

0.7745938628158845

In [73]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           0       0.80      0.76      0.78      1697
           1       0.77      0.85      0.81      1213
           2       0.75      0.72      0.74      1522

    accuracy                           0.77      4432
   macro avg       0.77      0.78      0.78      4432
weighted avg       0.77      0.77      0.77      4432

